# Mack RCNN Inference

In [17]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from object_detection.utils import label_map_util
import PIL.Image
import io
import pathlib

from collections import defaultdict
from io import StringIO
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [2]:
label_map = label_map_util.load_labelmap('labelmap.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [5]:
model = tf.saved_model.load(r'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/')

In [23]:
category_index = label_map_util.create_categories_from_labelmap(r'labelmap.pbtxt', use_display_name=True)

In [24]:
category_index

[{'id': 1, 'name': 'Cat'}, {'id': 2, 'name': 'Dog'}]

In [8]:
model.signatures['serving_default'].output_dtypes

{'refined_box_encodings': tf.float32,
 'proposal_boxes_normalized': tf.float32,
 'raw_detection_boxes': tf.float32,
 'num_detections': tf.float32,
 'detection_anchor_indices': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'anchors': tf.float32,
 'detection_masks': tf.float32,
 'num_proposals': tf.float32,
 'rpn_features_to_crop': tf.float32,
 'detection_scores': tf.float32,
 'rpn_objectness_predictions_with_background': tf.float32,
 'rpn_box_predictor_features': tf.float32,
 'class_predictions_with_background': tf.float32,
 'rpn_box_encodings': tf.float32,
 'detection_classes': tf.float32,
 'proposal_boxes': tf.float32,
 'image_shape': tf.float32,
 'box_classifier_features': tf.float32,
 'mask_predictions': tf.float32,
 'detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32,
 'final_anchors': tf.float32}

In [26]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    #print(num_detections)
    print(output_dict.keys())
    for key, value in output_dict.items():
        #print(key, value.numpy().shape, type(value))
        if len(value.shape) > 1:
            output_dict[key] = value[0][:num_detections]
        else:
            output_dict[key] = value[0]
    #output_dict = {key:value[0, :num_detections].numpy() 
    #               for key,value in output_dict.items()}
    #output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].numpy().astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict['detection_masks'], 
            output_dict['detection_boxes'],
            image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

In [27]:
def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    #print(output_dict)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'].numpy(),
        output_dict['detection_classes'],
        output_dict['detection_scores'].numpy(),
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)

    display(Image.fromarray(image_np))

In [18]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('dataset/downloads/extracted/TAR_GZ.robots.ox.ac.uk_vgg_pets_imageswMR1o1DWRq_DHWToagdXedb7P88RHpceK3WqG77VVwU.tar.gz/images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
len(TEST_IMAGE_PATHS)

7390

In [28]:
for image_path in TEST_IMAGE_PATHS:
    show_inference(model, image_path)
    break

UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node StatefulPartitionedCall/functional_1/conv2d/Conv2D}}]]
	 [[StatefulPartitionedCall/BatchMultiClassNonMaxSuppression_1/MultiClassNonMaxSuppression/Reshape_7/_532]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node StatefulPartitionedCall/functional_1/conv2d/Conv2D}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_signature_wrapper_43776]

Function call stack:
signature_wrapper -> signature_wrapper
